In [1]:
# SCRIPT TO GENERATE L CHANNEL IMAGE FROM RGB IMAGE FROM IMAGENET

import cv2
import os

# command to seach all subdir of image files and put it in single file
# find ./imagewang/train -type f -exec mv {} ./images_color \;

# specify the folder path
folder_path = './images_color'

#result forder for L images
result_path = './images_black'

# loop through all the files in the folder
for filename in os.listdir(folder_path):
    # check if the file is an image
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPEG"):
        # read the image
        img = cv2.imread(os.path.join(folder_path, filename))
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        L = lab[:,:,0]
        cv2.imwrite(f"{result_path}/{filename}", L)
        # do something with the image (e.g., display it)

In [2]:
color_words = [
    'red',
    'orange',
    'yellow',
    'green',
    'blue',
    'purple',
    'pink',
    'brown',
    'gray',
    'black',
    'white',
    'silver',
    'gold',
    'copper',
    'bronze',
    'cream',
    'beige',
    'ivory',
    'khaki',
    'navy',
    'turquoise',
    'teal',
    'aqua',
    'coral',
    'salmon',
    'lavender',
    'peach',
    'magenta',
    'fuchsia',
    'burgundy',
    'maroon',
    'olive',
    'lime',
    'mint',
    'cyan',
    'cobalt',
    'charcoal',
    'taupe',
    'mocha',
    'sage',
    'sand',
    'ruby',
    'emerald',
    'sapphire',
    'amethyst',
    'violet',
    'indigo',
    'crimson',
    'scarlet',
    'tangerine',
    'coral',
    'azure',
    'cerulean',
    'sienna',
    'umber',
    'terra cotta',
    'mauve',
    'lilac',
    'powder blue',
    'periwinkle',
    'cornflower blue',
    'baby blue',
    'sky blue',
    'steel gray',
    'rose',
    'blush',
    'champagne',
    'ivory black',
    'burnt sienna',
    'raw umber',
    'viridian',
    'caput mortuum',
    'cadmium red',
    'cadmium yellow',
    'cadmium orange',
    'cadmium green',
    'cadmium blue',
    'cerulean blue',
    'phthalo green',
    'phthalo blue',
    'quinacridone red',
    'quinacridone orange',
    'quinacridone yellow',
    'hansa yellow',
    'benzimidazolone yellow',
    'diarylide yellow',
    'pyrazolone orange',
    'naphthol red',
    'naphthol orange',
    'naphthol yellow',
    'permanent rose',
    'alizarin crimson',
    'ultramarine blue',
    'cobalt teal',
    'cerulean chromium',
    'manganese blue',
    'manganese violet',
    'dioxazine purple',
    'quinacridone violet',
    'raw sienna',
    'burnt umber',
    'ochre',
    'sienna brown',
    'sepia',
    'walnut',
    'mahogany',
    'ebony',
    'ivory black',
    'lamp black',
    'payne\'s gray',
    'davy\'s gray',
    'titanium white',
    'zinc white',
    'yellow ochre',
    'goldenrod',
    'saffron',
    'tawny',
    'russet',
    'ginger',
    'cinnamon',
    'mocha brown',
    'taupe gray',
    'ash gray',
    'smoke gray',
    'charcoal gray',
    'shadow gray',
    'misty gray',
    'fog gray',
    'steel blue',
    'prussian blue',
    'midnight blue',
    'navy blue',
    'royal blue',
    'baby blue',
    'powder blue',
    'periwinkle blue',
    'cornflower blue',
    'sky blue',
    'cerulean blue',
    'azure blue',
    'sapphire blue',
    'indigo blue',
    'violet blue',
    'magenta pink',
    'fuchsia pink',
    'rose pink',
    'blush pink',
    'coral pink',
    'salmon pink',
    'peach pink',
    'mauve pink',
    'lilac pink',
    'lavender pink',
    'pastel pink',
    'bubblegum pink',
    'hot pink',
    'neon pink',
    'bright pink',
    'deep pink',
    'rich pink',
    'vibrant pink',
    'magenta red',
    'fuchsia red',
    'crimson red',
    'scarlet red',
    'ruby red',
    'garnet red',
    'burgundy red',
    'maroon red',
    'claret red',
    'wine red',
    'mauve red',
    'lilac red'
]

color_set = set(color_words)

In [ ]:
# CAPTIONING IMAGES USING BLIP MODEL
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import os
import json

device = "cuda"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)


target_path = "../../DataSet/images_color"
source_path = "../../DataSet/images_black"

for filename in os.listdir(target_path):
    data = []
    img_url = f'{target_path}/{filename}' 
    raw_image = Image.open(img_url).convert('RGB')

    #open json file to dump data into in append mode
    f = open("prompt.json", "a")

    # conditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to(device)

    out = model.generate(**inputs)

    # have to do preprocessing based on color words and 40% of remaining captions iwth no color
    # 60% of the non color ones are for unconditional captions
    cap = processor.decode(out[0], skip_special_tokens=True)
    data.append({"source": f"{source_path}/{filename}", 
                 "target": f"{target_path}/{filename}", 
                 "prompt": cap})
    
    for item in data:
        json.dump(item, f)
        f.write('\n')

In [7]:
# HAVE TO MODIFY THIS BASED ON OUR DATA
import json
import cv2
import numpy as np
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('./prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread(source_filename)
        target = cv2.imread(target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0
        return dict(jpg=target, txt=prompt, hint=source)
    
dataset = MyDataset()
print(len(dataset))
temp = dataset[3]
print(temp)

6
{'jpg': array([[[ 0.16078436,  0.27058828, -0.31764704],
        [ 0.35686278,  0.41960788, -0.16862744],
        [ 0.00392163,  0.06666672, -0.23921567],
        ...,
        [ 0.54509807,  0.6       ,  0.54509807],
        [ 0.4666667 ,  0.54509807,  0.48235297],
        [ 0.39607847,  0.47450984,  0.41176474]],

       [[-0.4823529 , -0.44313723, -0.69411767],
        [-0.46666664, -0.372549  , -0.6862745 ],
        [ 0.09803927,  0.2313726 , -0.49019605],
        ...,
        [ 0.6       ,  0.69411767,  0.6       ],
        [ 0.5529412 ,  0.6627451 ,  0.56078434],
        [ 0.4901961 ,  0.6       ,  0.49803925]],

       [[-0.23137254, -0.20784312, -0.27843136],
        [-0.44313723, -0.47450978, -0.56078434],
        [ 0.05098045, -0.01176471, -0.29411763],
        ...,
        [ 0.56078434,  0.6156863 ,  0.5529412 ],
        [ 0.5294118 ,  0.60784316,  0.5372549 ],
        [ 0.5529412 ,  0.6313726 ,  0.56078434]],

       ...,

       [[-0.1372549 ,  0.26274514,  0.2941177 ],
 